<a href="https://colab.research.google.com/github/rpjena/random_matrix/blob/main/stock_residuals_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residuals & response functions via projection and exponential memory

We treat a panel of $N$ stock returns $R_i(t)$ observed at $T$ dates as a
vector field living on the time lattice.  The market return $R_m(t)$ is the
dominant collective mode.  Two operations:

1. **Project out the market mode** — the residual
   $\varepsilon_i(t) = R_i(t) - \alpha_i - \beta_i R_m(t)$
   is the fluctuation orthogonal to the market.  We solve for
   $(\alpha_i, \beta_i)$ via `scipy.linalg.lstsq`.

2. **Measure the response function** — for a PnL signal $P^{(r)}(t)$
   labelled by date $r$, the response of stock $i$ is
   $\beta_i^{(r)} = \operatorname{Cov}_w\!\bigl[P^{(r)},\,\varepsilon_i\bigr]
   \big/ \operatorname{Var}_w\![\varepsilon_i]$,
   where the exponential kernel $w_k = 2^{-k/h}$ (halflife $h = 21\text{d}$)
   implements finite memory, computed natively with `pandas.DataFrame.ewm`.

In [ ]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy import stats
import matplotlib.pyplot as plt

---
## I.  Project out the market mode

For the full panel at once, solve the overdetermined system
$\mathbf{X}\,\boldsymbol{\theta}_i = \mathbf{R}_i$
where $\mathbf{X} = [\mathbf{1} \;|\; \mathbf{R}_m]$ is the $(T \times 2)$
design matrix and $\boldsymbol{\theta}_i = (\alpha_i,\, \beta_i)^\top$.

`scipy.linalg.lstsq` uses the SVD internally — no need to form or invert
$\mathbf{X}^\top\mathbf{X}$, and it is numerically stable even when the
market has near-zero variance on some sub-interval.

In [ ]:
class StockResiduals:
    """
    Project out the market mode from a (T x N) return panel.

    R_i(t) = alpha_i + beta_i * R_m(t) + eps_i(t)

    Uses scipy.linalg.lstsq (SVD-based) for numerical stability.
    NaN rows are excluded per-stock; residuals inherit the original
    NaN mask so no information is fabricated.
    """

    def __init__(self, R, R_m, min_obs=30):
        T, N = R.shape
        assert len(R_m) == T, f"dim mismatch: {T} vs {len(R_m)}"

        self._R = R
        self._R_m = R_m
        self.min_obs = min_obs

        self.alphas, self.betas, self.r_squared, self.residuals, self.obs_count = (
            self._project()
        )

    # ------------------------------------------------------------------
    def _project(self):
        Y = self._R.values.astype(np.float64)       # (T, N)
        x = self._R_m.values.astype(np.float64)      # (T,)
        T, N = Y.shape
        cols, idx = self._R.columns, self._R.index

        # output buffers — NaN by default
        alpha = np.full(N, np.nan)
        beta  = np.full(N, np.nan)
        r2    = np.full(N, np.nan)
        eps   = np.full_like(Y, np.nan)
        nobs  = np.zeros(N, dtype=int)

        mkt_ok = ~np.isnan(x)
        stk_nan = np.isnan(Y)

        # --- batch path: stocks fully observed where market is valid ---
        has_gap = stk_nan[mkt_ok].any(axis=0)        # (N,)
        batch = ~has_gap

        if batch.sum() > 0:
            m = mkt_ok
            Xb = np.column_stack([np.ones(m.sum()), x[m]])   # (T', 2)
            Yb = Y[np.ix_(m, batch)]                         # (T', n_batch)
            nobs[batch] = m.sum()

            if m.sum() >= self.min_obs:
                # lstsq returns (coeffs, residuals_ss, rank, sv)
                theta, _, _, _ = linalg.lstsq(Xb, Yb)
                alpha[batch] = theta[0]
                beta[batch]  = theta[1]

                res = Yb - Xb @ theta
                eps[np.ix_(m, batch)] = res

                ss_res = np.sum(res ** 2, axis=0)
                ss_tot = np.sum((Yb - Yb.mean(axis=0)) ** 2, axis=0)
                r2[batch] = np.where(ss_tot > 0, 1 - ss_res / ss_tot, np.nan)

        # --- per-stock path: sparse columns ---
        for j in np.where(has_gap)[0]:
            ok = mkt_ok & ~stk_nan[:, j]
            t = ok.sum()
            nobs[j] = t
            if t < self.min_obs:
                continue

            Xj = np.column_stack([np.ones(t), x[ok]])
            yj = Y[ok, j]
            theta_j, _, _, _ = linalg.lstsq(Xj, yj)

            alpha[j] = theta_j[0]
            beta[j]  = theta_j[1]

            res_j = yj - Xj @ theta_j
            eps[ok, j] = res_j

            ss_res = np.sum(res_j ** 2)
            ss_tot = np.sum((yj - yj.mean()) ** 2)
            r2[j] = (1 - ss_res / ss_tot) if ss_tot > 0 else np.nan

        return (
            pd.Series(alpha, index=cols, name='alpha'),
            pd.Series(beta,  index=cols, name='beta'),
            pd.Series(r2,    index=cols, name='r_squared'),
            pd.DataFrame(eps, index=idx, columns=cols),
            pd.Series(nobs,  index=cols, name='obs_count'),
        )

    # ------------------------------------------------------------------
    def summary(self):
        return pd.DataFrame({
            'alpha': self.alphas, 'beta': self.betas,
            'r_squared': self.r_squared, 'obs_count': self.obs_count,
        })

---
## II.  Synthetic data — inject disorder, verify recovery

Generate $T = 2000$ dates, $N = 5000$ stocks with known couplings
$\beta_i \in [0.5, 1.8]$.  Sprinkle 5 % NaN ("missing measurements")
and check that the SVD-based projector recovers the true couplings.

In [ ]:
rng = np.random.default_rng(42)

T, N = 2000, 5000
beta_true  = rng.uniform(0.5, 1.8, N)
alpha_true = rng.normal(1e-4, 5e-4, N)
sigma_mkt, sigma_eps = 0.01, 0.02           # market vol, idio vol

R_m = rng.normal(5e-4, sigma_mkt, T)        # market field
noise = rng.normal(0, sigma_eps, (T, N))
R = alpha_true + np.outer(R_m, beta_true) + noise

dates = pd.date_range('2017-01-01', periods=T, freq='B')
tickers = [f'S{i:04d}' for i in range(N)]

R_df = pd.DataFrame(R, index=dates, columns=tickers)
R_m_sr = pd.Series(R_m, index=dates, name='MKT')

# disorder: random NaN, market gaps, one fully dark stock
R_df[rng.random((T, N)) < 0.05] = np.nan
R_m_sr.iloc[10:15] = np.nan
R_df['S0000'] = np.nan

print(f'R: {R_df.shape},  NaN frac = {R_df.isna().mean().mean():.3f}')
print(f'R_m: {R_m_sr.shape},  NaN count = {R_m_sr.isna().sum()}')

In [ ]:
proj = StockResiduals(R_df, R_m_sr)

print('=== first 10 couplings ===')
print(proj.summary().head(10))

In [ ]:
# recovery check: compare estimated vs true beta
ok = proj.obs_count >= 30
delta_beta = proj.betas[ok].values - beta_true[ok.values]

print(f'resolved stocks: {ok.sum()} / {N}')
print(f'delta_beta  —  mu = {delta_beta.mean():.2e},  sigma = {delta_beta.std():.4f}')
print(f'<R^2> = {proj.r_squared[ok].mean():.4f}')
print(f'S0000 (dark): beta = {proj.betas["S0000"]},  nobs = {proj.obs_count["S0000"]}')

In [ ]:
# scatter: estimated vs true coupling constant
fig, axes = plt.subplots(1, 2, figsize=(11, 4))

ax = axes[0]
ax.scatter(beta_true[ok.values], proj.betas[ok].values, s=0.3, alpha=0.4)
ax.plot([0.4, 1.9], [0.4, 1.9], 'k--', lw=0.8)
ax.set(xlabel=r'$\beta_i^{\mathrm{true}}$', ylabel=r'$\hat\beta_i$',
       title='coupling recovery')

ax = axes[1]
ax.hist(delta_beta, bins=80, density=True, alpha=0.7, color='steelblue')
# overlay Gaussian with same sigma
xg = np.linspace(-0.15, 0.15, 300)
ax.plot(xg, stats.norm.pdf(xg, 0, delta_beta.std()), 'k-', lw=1.2)
ax.set(xlabel=r'$\hat\beta_i - \beta_i^{\mathrm{true}}$',
       ylabel='density', title=r'estimation error ($\approx$ Gaussian)')

plt.tight_layout()
plt.show()

---
## III.  Response function $\beta_i^{(r)}$ with exponential memory

For each reference date $r$ and stock $i$ we measure the linear response
of a PnL signal $P^{(r)}(t)$ to the residual fluctuation $\varepsilon_i(t)$:

$$
\beta_i^{(r)} = \frac{\operatorname{Cov}_w\!\bigl[P^{(r)},\, \varepsilon_i\bigr]}
                      {\operatorname{Var}_w\![\varepsilon_i]}
\qquad\text{with}\qquad
w_k = 2^{-k/h},\; h = 21\text{d}
$$

The exponential kernel is the *retarded Green's function* of the
discrete-time equation $\dot{\beta} = -\lambda\, \beta + \eta(t)$ with
$\lambda = \ln 2 / h$.  It forgets the past on a timescale $\tau = h / \ln 2
\approx 30$ days.

We compute this **entirely with `pandas.ewm`**: for each date $r$,
align $P^{(r)}$ with the residual panel, then read off the EWM
second moments at the last time step.

In [ ]:
class PnLBeta:
    """
    EWM response function: beta_i(r) = Cov_w[P(r), eps_i] / Var_w[eps_i].

    Uses pandas.DataFrame.ewm (halflife h) to compute the weighted
    second moments — no hand-rolled weight arrays needed.

    Parameters
    ----------
    eps : DataFrame, (N x T)
        Residuals.  Rows = stocks, columns = dates.
    pnl : DataFrame, (T x T)
        Columns = dates.  pnl[r] is the PnL vector for reference date r.
    halflife : int
        Exponential memory in business days.  Default 21 (~1 month).
    min_obs : int
        Minimum jointly non-NaN observations.  Default 30.
    """

    def __init__(self, eps, pnl, halflife=21, min_obs=30):
        common = eps.columns.intersection(pnl.columns)
        if common.empty:
            raise ValueError("no overlapping dates")

        self.eps = eps
        self.pnl = pnl
        self.halflife = halflife
        self.min_obs = min_obs

        self.betas_raw, self.r_squared = self._measure()
        self.betas = self.betas_raw.ffill(axis=1)     # carry forward

    # ------------------------------------------------------------------
    def _measure(self):
        """
        For each date r, compute the EWM response beta_i(r) using
        pandas ewm machinery.  The key identity:

            Cov_w[P, eps_i] = E_w[P * eps_i] - E_w[P] * E_w[eps_i]
            Var_w[eps_i]    = E_w[eps_i^2]   - E_w[eps_i]^2

        pandas .ewm(halflife=h).mean() computes the running weighted
        average with the correct normalisation.  We only need the
        terminal value (iloc[-1]).
        """
        eps_T = self.eps.T                 # (T x N) — time on rows for ewm
        stocks = self.eps.index
        date_cols = self.eps.columns
        N, T_cols = len(stocks), len(date_cols)
        h = self.halflife

        beta_arr = np.full((N, T_cols), np.nan)
        r2_arr   = np.full((N, T_cols), np.nan)

        for c, date_r in enumerate(date_cols):
            if date_r not in self.pnl.columns:
                continue

            p = self.pnl[date_r].dropna()           # PnL signal for date r
            common = p.index.intersection(date_cols)
            L = len(common)
            if L < self.min_obs:
                continue

            # align PnL and residuals on common dates, time on axis=0
            p_c = p.loc[common]                       # (L,)
            e_c = eps_T.loc[common]                   # (L x N)

            # -- EWM moments via pandas --
            ewm_kw = dict(halflife=h, min_periods=self.min_obs)

            mu_e = e_c.ewm(**ewm_kw).mean().iloc[-1]                     # (N,)
            mu_p = p_c.ewm(**ewm_kw).mean().iloc[-1]                     # scalar

            # E_w[P * eps]
            pe = e_c.multiply(p_c, axis=0)
            mu_pe = pe.ewm(**ewm_kw).mean().iloc[-1]                     # (N,)

            # E_w[eps^2]
            mu_e2 = (e_c ** 2).ewm(**ewm_kw).mean().iloc[-1]            # (N,)

            # skip if ewm returned NaN (fewer than min_periods non-NaN)
            if np.isnan(mu_p):
                continue

            cov = mu_pe - mu_e * mu_p                # Cov_w[P, eps_i]
            var = mu_e2 - mu_e ** 2                  # Var_w[eps_i]

            good = var > 0
            b = np.where(good, cov / var, np.nan)
            a = mu_p - b * mu_e
            beta_arr[:, c] = b

            # weighted R^2: 1 - Var_w[residual] / Var_w[P]
            # where residual of the response regression = P - a - b * eps
            fitted = a + b * e_c.values               # (L x N)
            p_vals = p_c.values[:, np.newaxis]        # (L x 1)
            resid_sq = (p_vals - fitted) ** 2         # (L x N)

            # compute EWM of squared residual at terminal time
            resid_sq_df = pd.DataFrame(resid_sq, index=common, columns=stocks)
            ss_res = resid_sq_df.ewm(**ewm_kw).mean().iloc[-1]  # (N,)

            p_dm_sq = (p_c - mu_p) ** 2
            ss_tot = p_dm_sq.ewm(**ewm_kw).mean().iloc[-1]      # scalar

            r2_arr[:, c] = np.where(
                (ss_tot > 0) & good, 1 - ss_res.values / ss_tot, np.nan
            )

        return (
            pd.DataFrame(beta_arr, index=stocks, columns=date_cols),
            pd.DataFrame(r2_arr,   index=stocks, columns=date_cols),
        )

    # ------------------------------------------------------------------
    def summary_at(self, date):
        return pd.DataFrame({
            'beta': self.betas[date], 'r_squared': self.r_squared[date],
        })

---
## IV.  Measure the response on synthetic data

In [ ]:
eps_NxT = proj.residuals.T                            # (N x T)
print(f'eps: {eps_NxT.shape}')

# synthetic PnL: (T x T), columns = dates
rng2 = np.random.default_rng(99)
pnl_data = rng2.normal(0, 0.01, (T, T))
pnl_data[rng2.random((T, T)) < 0.03] = np.nan        # 3 % disorder
pnl_df = pd.DataFrame(pnl_data, index=dates, columns=dates)

print(f'PnL: {pnl_df.shape},  NaN frac = {pnl_df.isna().mean().mean():.3f}')

In [ ]:
resp = PnLBeta(eps_NxT, pnl_df, halflife=21, min_obs=30)

print(f'beta_raw : {resp.betas_raw.shape}')
print(f'beta     : {resp.betas.shape}')
print(f'R^2      : {resp.r_squared.shape}')
print(f'halflife : {resp.halflife} d')
print()
print('--- raw betas (5 stocks x last 5 dates) ---')
print(resp.betas_raw.iloc[:5, -5:])
print()
print('--- forward-filled betas (5 stocks x last 5 dates) ---')
print(resp.betas.iloc[:5, -5:])

In [ ]:
# diagnostics at the last date
snap = resp.summary_at(dates[-1])
print(f'=== snapshot at {dates[-1].date()} (first 10) ===')
print(snap.head(10))
print()
print(f'S0000 (dark stock) beta = {resp.betas.loc["S0000"].iloc[-1]}')
print(f'NaN:  raw = {resp.betas_raw.isna().sum().sum()}, '
      f'after ffill = {resp.betas.isna().sum().sum()}')

In [ ]:
# distribution of terminal betas (excluding dark stock)
b_last = resp.betas.iloc[:, -1].dropna()

fig, ax = plt.subplots(figsize=(6, 3.5))
ax.hist(b_last, bins=80, density=True, alpha=0.7, color='coral')
mu, sig = b_last.mean(), b_last.std()
xg = np.linspace(mu - 4*sig, mu + 4*sig, 300)
ax.plot(xg, stats.norm.pdf(xg, mu, sig), 'k-', lw=1.2)
ax.set(xlabel=r'$\beta_i^{(r_{\max})}$', ylabel='density',
       title=f'EWM response distribution at terminal date (h={resp.halflife}d)')
ax.axvline(0, color='grey', ls='--', lw=0.7)
plt.tight_layout()
plt.show()

---
## V.  Sanity check: halflife → ∞ should recover OLS

In the limit $h \to \infty$ the kernel becomes flat and the EWM
estimator reduces to the equal-weight sample covariance, i.e. ordinary
least squares.  We verify on a small slice.

In [ ]:
# pick a small sub-panel for speed
n_sub = 50
eps_sub = eps_NxT.iloc[:n_sub, :200]
pnl_sub = pnl_df.iloc[:200, :200]

resp_flat = PnLBeta(eps_sub, pnl_sub, halflife=100_000, min_obs=30)
resp_ewm  = PnLBeta(eps_sub, pnl_sub, halflife=21,      min_obs=30)

# also compute OLS betas by hand via scipy for one date
date_r = eps_sub.columns[100]
p = pnl_sub[date_r].dropna()
common = p.index.intersection(eps_sub.columns)
p_c = p.loc[common].values
e_c = eps_sub[common].values.T           # (L x n_sub)

# OLS via scipy: theta = lstsq([1 | e_i], p) for each stock
ols_betas = np.full(n_sub, np.nan)
for j in range(n_sub):
    mask = ~np.isnan(e_c[:, j]) & ~np.isnan(p_c)
    if mask.sum() < 30:
        continue
    X_j = np.column_stack([np.ones(mask.sum()), e_c[mask, j]])
    theta_j, _, _, _ = linalg.lstsq(X_j, p_c[mask])
    ols_betas[j] = theta_j[1]

flat_betas = resp_flat.betas[date_r].values
ewm_betas  = resp_ewm.betas[date_r].values

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

ax = axes[0]
ax.scatter(ols_betas, flat_betas, s=20, alpha=0.7)
lim = [np.nanmin(ols_betas)*1.1, np.nanmax(ols_betas)*1.1]
ax.plot(lim, lim, 'k--', lw=0.8)
ax.set(xlabel=r'$\beta_i$ (scipy lstsq)', ylabel=r'$\beta_i$ (EWM, $h=10^5$)',
       title=r'$h\to\infty$ limit $\approx$ OLS')

ax = axes[1]
ax.scatter(ols_betas, ewm_betas, s=20, alpha=0.7, color='coral')
ax.plot(lim, lim, 'k--', lw=0.8)
ax.set(xlabel=r'$\beta_i$ (scipy lstsq)', ylabel=r'$\beta_i$ (EWM, $h=21$)',
       title=r'finite memory $h=21$ d')

plt.tight_layout()
plt.show()

diff = flat_betas - ols_betas
print(f'h→∞ vs OLS:  max|delta| = {np.nanmax(np.abs(diff)):.2e}  (should be ~0)')